In [ ]:
% Toolbox Setup
cd ../
startup

# SNR in Hyperpolarized MRI

SNR in hyperpolarized MRI still obeys the fundamental SNR relationship for MRI

$$SNR \propto f_{seq}\ \mathrm{Voxel\ Volume}\ \sqrt{T_{acq}}$$

where SNR is proportional to transverse magnetization generated by a pulse sequences, $f_{seq}$, the voxel volume, and the total measurement time, $T_{acq}$.

However, we have a limited amount of magnetization available for hyperpolarized MRI that we may also want to spread across time for dynamic measurements of kinetics.  To analyze the choice of flip angle, we will assume (1) that the voxel volume is fixed; and (2) that the readout time for each TR is fixed (e.g. limited by $T_2^*$ decay).

Below will examine the total SNR achieved (e.g. if all images were added together) under the following assumptions
* RF pulses alone
* RF pulses and T1 decay
* RF pulses, T1 decay, and metabolic conversion
* RF pulses, T1 decay, metabolic conversion and bolus delivery

For each set of assumptions, we will examine the impact of the
* Flip angle choice
* Scan duration
* Repetition time (TR)


## Effect of RF pulses alone

Each RF pulse applied we result in some usage of the unrecoverable hyperpolarized magnetization because of T2 and T2* decay during the readout.  So for simplicity we assume a spoiled gradient-echo signal model.

### Constant flip angles




### Variable flip angles

If we used a variable flip angle to distribute the magnetization evenly across 



## Add T1 decay

Just T1 decay, then we want to measure as fast as possible




Number of measurements split - proper averaging

Number of scans
repetition time
(vs T1)
Fixed time, adjust flip angle (2d flip angle simulation)

VFA (no T1)
Start with just T1 (e.g. HP gas)
Add in metabolic conversion
Add in bolus

Balancing flip angles - Sule?  Can she copy something I do?

Optimal SNR - check flip angles un

In [ ]:
NTRs = 1:100;
flips = 5:5:90;

SNR_const = zeros(length(NTRs), length(flips));
SNR_vfa = zeros(length(NTRs), 1);

 
for INTRs = 1:length(NTRs)
    for Iflips = 1:length(flips)

        [mxy, mz] = hyperpolarized_mag_usage(flips(Iflips) * ones(1, NTRs(INTRs)));
        SNR_const(INTRs, Iflips) = sum(mxy) ./ sqrt(NTRs(INTRs));

    end
    flips_vfa = vfa_const_amp(NTRs(INTRs), pi/2);
    [mxy, mz] = hyperpolarized_mag_usage(flips_vfa);
    SNR_vfa = sum(mxy) ./ sqrt(NTRs(INTRs));
end

plot(NTRs,SNR_const, NTRs, SNR_vfa)
xlabel('Number of TRs'), ylabel('relative SNR')
colorbar

In [ ]:


% fixed
% vfa_opt_signal(N, exp(-TR / T1 )


In [ ]:
% Sample variable flip angle generation code
% 
% Author: Peder E. Z. Larson
%
% (c)2014-2015 The Regents of the University of California.
% All Rights Reserved.

%% example for pyruvate (substrate) and lactate (product) 

N = 12; % number of pulses
TR = 3; % repetition time
kPL = 0.05;  % conversion rate from pyruvate to lactate
R1P = 1/35; R1L = 1/30;  % relaxation rates of pyruvate and lactate
Minit = [1; 0.05];  % initial magnetization of Pyr, Lac

% flips_XXX - first row is pyruvate flip angles, second row is lactate flip
% angles

% maximum lactate SNR scheme 
%  (invariant with pyruvate flip angles, so adjust as needed to get 
%   sufficient pyruvate SNR)
flips_maxsnr = [vfa_const_amp(N, pi/2, exp(-TR * (R1P + kPL))); ...
         vfa_opt_signal(N, exp(-TR * R1L ))];

% T1-effective scheme
flips_t1eff = [vfa_const_amp(N, pi/2, exp(-TR * (R1P + kPL))); ...
         vfa_const_amp(N, pi/2, exp(-TR * (R1L - kPL)))];

% const-signal scheme - optimize for constant signal amplitude
%   with the chosen parameters this is not realizable, but increasing N or
%   the initial lactate magnetization, Minit(2), will result in a constant
%   amplitude solution
flips_constsignal = mvfa_const_amp(N, TR, kPL, [R1P R1L], Minit);

[Mxy_maxsnr, Mz_maxsnr] = simulate_Nsite_model(Minit, [R1P R1L], [kPL 0], flips_maxsnr, TR);
[Mxy_t1eff, Mz_t1eff] = simulate_Nsite_model(Minit, [R1P R1L], [kPL 0], flips_t1eff, TR);
[Mxy_constsignal, Mz_constsignal] = simulate_Nsite_model(Minit, [R1P R1L], [kPL 0], flips_constsignal, TR);

t = [0:N-1] * TR;
figure
subplot(221)
plot(t, flips_maxsnr(1,:)*180/pi,t, flips_t1eff(1,:)*180/pi, t, flips_constsignal(1,:)*180/pi)
title('Pyruvate'), legend('Max SNR', 'T1-effective', 'const-signal')
xlabel('time (s)'), ylabel('flip angle (degrees)')
subplot(222)
plot(t, flips_maxsnr(2,:)*180/pi,t, flips_t1eff(2,:)*180/pi, t, flips_constsignal(2,:)*180/pi)
title('Lactate')
xlabel('time (s)'), ylabel('flip angle (degrees)')
subplot(223)
plot(t, Mxy_maxsnr(1,:),t, Mxy_t1eff(1,:), t, Mxy_constsignal(1,:))
title('Pyruvate'), xlabel('time (s)'), ylabel('Simulated Signal')
subplot(224)
plot(t, Mxy_maxsnr(2,:),t, Mxy_t1eff(2,:), t, Mxy_constsignal(2,:))
title('Lactate')
xlabel('time (s)'), ylabel('Simulated Signal')

%%
validate = 1;

if validate
    flips_maxsnrnew = flips_maxsnr; flips_t1effnew = flips_t1eff; flips_constsignalnew = flips_constsignal;
    load test_data/vfa_example_data
    if max(abs(flips_maxsnrnew - flips_maxsnr)) > eps | max(abs(flips_t1effnew - flips_t1eff)) > eps | max(abs(flips_constsignalnew - flips_constsignal )) > eps
        error('Validation failed for variable flip angle example')
    else
        disp('Validation passed for variable flip angle example')
    end
end


In [ ]:
clear all
clc

fprintf('This scripts tests an approximate max lactate SNR flip schedule\n versus those derived using a full optimization solver\n')
fprintf('--------------------------\n')


% default parameters
N = 16; % number of pulses
TR = 3; % repetition time
kPL = 0.05;  % conversion rate from pyruvate to lactate
R1P = 1/35; R1L = 1/30;  % relaxation rates of pyruvate and lactate
Tinj = 10; %injection time

% flip angle schedule for approximately maximum lactate SNR to be validated
fprintf('The approximate max lactate SNR flip schedule is generated by:\n')
fprintf('   vfa_opt_signal(N, exp(-TR * R1L ))\n\n')
fprintf('Note that the approximate maximum SNR lactate flip angles only depend on the TR and lactate T1\n')
fprintf('This is truly maximized with pyruvate flip angles of zero, so to detect pyruvate we choose a\n')
fprintf('T1-effective schedule to provide sufficient SNR as:\n')
fprintf('   vfa_const_amp(N, pi/2, exp(-TR * (R1P + kPL)))\n\n')

flips_maxsnr = [vfa_const_amp(N, pi/2, exp(-TR * (R1P + kPL))); ...
         vfa_opt_signal(N, exp(-TR * R1L ))];

%% test approximate max lactate SNR flip schedule using full optimization solver

% R1P
fprintf('\nTest over various pyruvate relaxation rates:\n')
fprintf('\tmaximum difference between approximate solution and full optimization for\n')
for R1P_new = [1/10, 1/30, 1/50]
    flips_test = optimal_SNR_angles(N*TR,TR,kPL,[R1P_new, R1L],Tinj, flips_maxsnr(1,:));
    fprintf('\t\tT1pyurvate = %d s is %.4f degrees\n', 1/R1P_new, max(abs(flips_maxsnr(2,:) - flips_test(2,:)))*180/pi)
end

% compare for different R1L
fprintf('\nTest over various lactate relaxation rates:\n')
fprintf('\tmaximum difference between approximate solution and full optimization for\n')
for R1L_new = [1/10, 1/30, 1/50]
    flips_maxsnr_new = [vfa_const_amp(N, pi/2, exp(-TR * (R1P + kPL))); ...
         vfa_opt_signal(N, exp(-TR * R1L_new ))];

    flips_test = optimal_SNR_angles(N*TR,TR,kPL,[R1P, R1L_new],Tinj, flips_maxsnr_new(1,:));
    fprintf('\t\tT1lactate = %d s is %.4f degrees\n', 1/R1L_new, max(abs(flips_maxsnr_new(2,:) - flips_test(2,:)))*180/pi)
end

% kPL
fprintf('\nTest over various conversion rates:\n')
fprintf('\tmaximum difference between approximate solution and full optimization for\n')
for kPL_new = [.01 .05 .1]
    flips_test = optimal_SNR_angles(N*TR,TR,kPL_new,[R1P, R1L],Tinj, flips_maxsnr(1,:));
    fprintf('\t\tkPL = %.2f 1/s is %.4f degrees\n', kPL_new, max(abs(flips_maxsnr(2,:) - flips_test(2,:)))*180/pi)
end

% compare for different TR
fprintf('\nTest over various TR:\n')
fprintf('\tmaximum difference between approximate solution and full optimization for\n')
for TR_new = [1 3 6]
    flips_maxsnr_new = [vfa_const_amp(N, pi/2, exp(-TR_new * (R1P + kPL))); ...
         vfa_opt_signal(N, exp(-TR_new * R1L ))];

    flips_test = optimal_SNR_angles(N*TR_new,TR_new,kPL,[R1P, R1L],Tinj, flips_maxsnr_new(1,:));
    fprintf('\t\tTR = %d s is %.4f degrees\n', TR_new, max(abs(flips_maxsnr_new(2,:) - flips_test(2,:)))*180/pi)
end


% substrate_flips
fprintf('\nTest over various pyruvate flip schedules (using T1-effective with range of T1eff):\n')
fprintf('\tmaximum difference between approximate solution and full optimization for\n')
for R1eff_new = [-.05 0 .05]
    flips_test = optimal_SNR_angles(N*TR,TR,kPL,[R1P, R1L],Tinj, vfa_const_amp(N, pi/2, exp(-TR * R1eff_new)));
    fprintf('\t\tT1eff = %.2f s is %.4f degrees\n', 1/R1eff_new, max(abs(flips_maxsnr(2,:) - flips_test(2,:)))*180/pi)
end

% compare for different N
fprintf('\nTest over various number of pulses:\n')
fprintf('\tmaximum difference between approximate solution and full optimization for\n')
for N_new = [8 16 32]
    flips_maxsnr_new = [vfa_const_amp(N_new, pi/2, exp(-TR * (R1P + kPL))); ...
         vfa_opt_signal(N_new, exp(-TR * R1L ))];

    flips_test = optimal_SNR_angles(N_new*TR,TR,kPL,[R1P, R1L],Tinj, flips_maxsnr_new(1,:));
    fprintf('\t\tN pulses = %d is %.4f degrees\n', N_new, max(abs(flips_maxsnr_new(2,:) - flips_test(2,:)))*180/pi)
end

% Tinj
% This is similar to testing over different initial magnetizations (Minit)
% used in other code in this project
fprintf('\nTest over various injection durations:\n')
fprintf('\tmaximum difference between approximate solution and full optimization for\n')
for Tinj_new = [2 10 20]
    flips_test = optimal_SNR_angles(N*TR,TR,kPL,[R1P, R1L],Tinj_new, flips_maxsnr(1,:));
    fprintf('\t\tTinj = %d s is %.4f degrees\n', Tinj_new, max(abs(flips_maxsnr(2,:) - flips_test(2,:)))*180/pi)
end


%% determine optimal SNR in free parameter dimensions
% based on above results, only remaining free parameters are:
%    total time and TR
% and optimal results may depend on T1 of lactate.
% Below are some brute force simulations to try and solve amongst these
% free parameters.
% Not finished yet, no comparison across TR:
%  (I think the error is due to scaling of input function in 'trajectories
%   sub-function of optimal_SNR_angles - PEZL)

clear S_noinj S_inj

TR = 2;

T = [10:2:60];
R1L = 1./[15:5:40];
for IT = 1:length(T)
    for IR1L = 1:length(R1L)
        % using negligible pyruvate flip angles

        % Tinj = 2 is like starting after injection
        [flips, Mxy, Mz] = optimal_SNR_angles(T(IT), TR, kPL, [R1P R1L(IR1L)], 2, pi/100*ones(floor(T(IT)/TR),1));
        S_noinj(IT, IR1L) = sum(Mxy(2,:)) / sqrt(size(Mxy,2));

        % Tinj = 10 is like a typical, 10-15 s injection time
        [flips, Mxy, Mz] = optimal_SNR_angles(T(IT), TR, kPL, [R1P R1L(IR1L)], 10, pi/100*ones(floor(T(IT)/TR),1));
        S_inj(IT, IR1L) = sum(Mxy(2,:)) / sqrt(size(Mxy,2));

    end
end

figure
subplot(1,2,1)
mesh(1./R1L, T, S_noinj)
axis tight
xlabel('T_{1.lactate}'), ylabel('Total time'), zlabel('Total SNR')
title('Start after injection')
view([-65 12])

subplot(1,2,2)
mesh(1./R1L, T, S_inj)
axis tight
xlabel('T_{1.lactate}'), ylabel('Total time'), zlabel('Total SNR')
title('Start during injection')
view([-65 12])


## References

Zhao

Nagashima